# Описание формата исходных данных

assets.json - описание активов модели

Формат описания:  
title - наименование актива, должно совпадать с используемым в матрице уязвимости и описании FT  
valuable - признак ценности актива (сейчас не используется)  
TTR - время восстановления актива (должно рассчитываться при подготовке входных данных симуляции, здесь - задается одним числом)  
Network - список логических сегментов актива (массив строк)  
Local - список локлаьных сегментов актива  
Physical - список физических сегментов актива  

FT.json - описание дерева неисправностей

Корневой узел должен быть ровно один  
каждый узел содержит атрибуты:  
name - имя узла, совпадающее с атрибутом title в описании активов (assets.json)  
impact - величина ущерба для узла дерева (по умолчанию - 0)  
node_type - тип узла. Допустимые варианты:  
basic - простое событие (лист дерева)  
root - корневое событие (начало дерева)  
and - логическое И  
or - логическое ИЛИ  
k from n - мажоритарное ИЛИ. Для типа "k from n" обязательно должен быть указан атрибут k (иначе будет принят k = 1)  
childs - дочерние атрибуты. Не должно быть у атрибутов типа "basic" (проверка не реализована)  


# Импорт библиотек

In [1]:
import os
os.path.curdir


'.'

In [2]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
import json
import random
import time
import oct_read
from scipy.stats import rankdata

# Чтение исходных данных

In [3]:
#Data_Format = 'Native'
Data_Format = 'Octonica'
nb = True

if Data_Format == 'Native':
    import os
    #os.chdir('D:\Google\work\ПОАР\Model\simattack')
    input_data = 'split-example-sub2'
elif Data_Format == 'Octonica':
    json_name = '.\\Octonica\\файл_для_отладки_симуляции_23_12_2021.json'
    sub_name = ['Подсистема1', 'Подсистема2']
    input_data = 'Octonica'
    #первый в списке
    intruder_id = 'fdaec6d3-e8f3-4206-b71d-01774404c129'
    #второй в списке
    #intruder_id = '603663d5-b312-46b3-9916-ab7d67474b00'
    









In [4]:
if Data_Format == 'Native':
    assets = pd.read_json('.\\' + input_data + '\\assets.json',orient='records')
    #задаем контекст
    cols = ['a_id', 'AV', 'PR', 'I']
    Ctx_0 = pd.DataFrame(columns = cols)
    #___________________контекст для example1___________________________________________
    #Ctx_0 = Ctx_0.append({'a_id' : 4, 'AV' : 0, 'PR' : 0, 'I' : 0}, ignore_index = True)
    #Ctx_0 = Ctx_0.append({'a_id' : 9, 'AV' : np.inf, 'PR' : 0, 'I' : 0}, ignore_index = True)
    #___________________контекст для minimal___________________________________________
    #Ctx_0 = Ctx_0.append({'a_id' : 0, 'AV' : 0, 'PR' : 0, 'I' : 0}, ignore_index = True)
    #Ctx_0 = Ctx_0.append({'a_id' : 1, 'AV' : 0, 'PR' : 0, 'I' : 0}, ignore_index = True)
    #___________________контекст для split-example___________________________________________
    #Ctx_0 = Ctx_0.append({'a_id' : 4, 'AV' : 0, 'PR' : 0, 'I' : 0}, ignore_index = True)
    #Ctx_0 = Ctx_0.append({'a_id' : 9, 'AV' : np.inf, 'PR' : 0, 'I' : 0}, ignore_index = True)
    #Ctx_0 = Ctx_0.append({'a_id' : 5, 'AV' : 1, 'PR' : 0, 'I' : 0}, ignore_index = True)
    #___________________контекст для split-example-sub1___________________________________________
    #Ctx_0 = Ctx_0.append({'a_id' : 4, 'AV' : 0, 'PR' : 0, 'I' : 0}, ignore_index = True)
    #Ctx_0 = Ctx_0.append({'a_id' : 5, 'AV' : np.inf, 'PR' : 0, 'I' : 0}, ignore_index = True)
    #___________________контекст для split-example-sub2___________________________________________
    Ctx_0 = Ctx_0.append({'a_id' : 0, 'AV' : 1, 'PR' : 0, 'I' : 0}, ignore_index = True)
    #Ctx_0 = Ctx_0.append({'a_id' : 4, 'AV' : np.inf, 'PR' : 0, 'I' : 0}, ignore_index = True)
    vulner = pd.read_excel('.\\' + input_data + '\\data.xlsx',sheet_name='Vulnerabilities', index_col='id')
    counter_m = pd.read_excel('.\\' + input_data + '\\data.xlsx',sheet_name='C-Measures', index_col='id')
    prevent_matrix = pd.read_excel('.\\' + input_data + '\\data.xlsx',sheet_name='Measures-Vuln-P', index_col='id', dtype=object)
    detect_matrix = pd.read_excel('.\\' + input_data + '\\data.xlsx',sheet_name='Measures-Vuln-D', index_col='id', dtype=object)
    vuln_matrix = pd.read_excel('.\\' + input_data + '\\data.xlsx',sheet_name='Assets-Vuln', index_col='title', dtype=object)
    #потенциал нарушителя
    k = 1
    #перевод обозначений AV и PR в числа
    vulner.loc[vulner.AV == 'P','AV'] = -np.inf
    vulner.loc[vulner.AV == 'L','AV'] = 0
    vulner.loc[vulner.AV == 'A','AV'] = 1
    vulner.loc[vulner.AV == 'N','AV'] = 2

    vulner.loc[vulner.PR == 'N','PR'] = 0
    vulner.loc[vulner.PR == 'L','PR'] = 1
    vulner.loc[vulner.PR == 'H','PR'] = 2

    vulner.loc[vulner['s.AV'] == 'P','s.AV'] = np.inf
    vulner.loc[vulner['s.AV'] == 'L','s.AV'] = 0
    vulner.loc[vulner['s.AV'] == 'A','s.AV'] = 1
    vulner.loc[vulner['s.AV'] == 'N','s.AV'] = 2

    vulner.loc[vulner['s.PR'] == 'N','s.PR'] = 0
    vulner.loc[vulner['s.PR'] == 'L','s.PR'] = 1
    vulner.loc[vulner['s.PR'] == 'H','s.PR'] = 2
    #рассчитываем вероятность реализации уязвимости
    cont_counter_m = counter_m[counter_m['Период'] == 0]
    for (vuln, prob) in prevent_matrix.iteritems():
        vulner.at[vuln, 'P_v'] = (1 - prob[cont_counter_m.index]).prod() * k * vulner.at[vuln,'Prob']
    for (vuln, prob) in detect_matrix.iteritems():
        vulner.at[vuln, 'P_d'] = 1 - (1 - prob[cont_counter_m.index]).prod()
elif Data_Format == 'Octonica':
    data_j = oct_read.init_json(json_name)
    sub_id = oct_read.get_sub_id(data_j,sub_name)
    (assets_filtered, assets) = oct_read.get_assets_df(data_j,sub_id)
    Ctx_0 = oct_read.get_ctx0(data_j)
    Ctx_0 = Ctx_0.query('intruderId == @intruder_id')
    Ctx_0 = Ctx_0.drop(['intruderId'], axis = 1)
    (counter_m, vulners_assets_dict) = oct_read.get_vulners_cmeasures(data_j, assets_filtered, intruder_id)
    
assets['t_recover'] = 0
#периодические меры
t_measures = counter_m[counter_m['Период'] != 0]



D:\YandexDisk\work\ПОАР\Model\simattack\simattack\oct_read.py:218: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p_idx = list(counter_m.loc[p_idx][counter_m['Период'] == 0].index)


In [5]:
#создание словаря, где ключ - id контрмеры, а значение - порядковый номер соответствующей строки в DF контрмер
c_id_dict = dict(zip(list(counter_m.index),range(len(counter_m.index))))

vuln_df = pd.json_normalize(data_j['vulnerabilities'])
#создание словаря, где ключ - id уязвимости, а значение - порядковый номер соответствующей строки в DF уязвимостей
vuln_df = vuln_df.set_index('id')
v_id_dict = dict(zip(list(vuln_df.index),range(len(vuln_df.index))))

#создание словаря, где ключ - id актива, а значение - порядковый номер соответствующей строки в DF активов
a_id_dict = dict(zip(list(assets.index),range(len(assets.index))))


## Расчет значения $\tau$ для периодических мер

In [6]:
#заготовка матрицы ожидаемого времени обнаружения связки (акитв, уязвимость, контрмера (периодическая))
matrix_measures = np.full((len(a_id_dict), len(v_id_dict), len(c_id_dict)), np.inf)

In [7]:
#проходим по словарю уязвимостей (id актива, описание уязвимости)
for (a_df_id, item) in vulners_assets_dict.items():
    #вытаскиваем строку с промежуточным json для получения полной информации об уязвимостях
    row = assets_filtered.query("id == @a_df_id").iloc[0]
    #преобразуем json с описанием уязвимостей в DF
    v_df = pd.json_normalize(row['protectionMeasures'], record_path='vulnerabilities', max_level=0,
                                         meta=['protectionMeasureId'], errors='ignore')
    #если есть уязвимости у актива
    if not(v_df.empty):
        #выбираем все детектирующие меры
        v_df = v_df.query("canDetect")
        #проходим по детектирующим мерам
        for (row_id, vuln) in v_df.iterrows():
            
            #сохраняем значения id и индексы id в массиве
            v_df_id = vuln['id']
            v_id = v_id_dict[v_df_id]
            c_df_id = vuln['protectionMeasureId']
            c_id = c_id_dict[c_df_id]
            if c_df_id in t_measures.index:
                a_id = a_id_dict[a_df_id]
                #сохраняем значения вероятности обнаружения и периода
                p_d = t_measures.at[c_df_id,'P_d']
                T = t_measures.at[c_df_id,'Период']
                #если вероятность 1, то мы гарантирвоано обнаружим за одно применение меры (т.е. за время T)
                matrix_measures[a_id, v_id, c_id] = T
                if p_d < 1:
                    #если вероятность обнаружения менее 1, то считаем по схеме Бернулли
                    matrix_measures[a_id, v_id, c_id] = np.ceil(np.log(0.05)/np.log(1-p_d)) * T
                    
            
            
                
            
    
    

# Построение графа связности активов

In [8]:
Link_MG = nx.Graph()

In [9]:
%%time
assets['label'] = assets['title'] + '#' + assets.index.astype(str)
assets['shape'] = 'ellipse'
#Link_MG.add_nodes_from(list(assets.loc[:,'title':'valuable'].to_dict(orient='index').items()))
Link_MG.add_nodes_from(list(assets.to_dict(orient='index').items()))

Wall time: 4.99 ms


In [10]:
od_iter = OrderedDict()
od_iter['Physical'] = np.inf
od_iter['Network'] = 1
od_iter['Local'] = 0

In [11]:
#d_iter = {'Physical': np.inf, 'Network': 1,'Local' : 0}
for i in assets.index:
    for j in assets.index:
        if (i != j):
            for (domain, weight) in od_iter.items():
                if (len(set(assets.loc[i,domain]) & set(assets.loc[j,domain])) > 0):
                    Link_MG.add_edge(i,j,weight = weight)          
                    Link_MG[i][j]['label'] = weight


In [12]:
Link_MG.edges

EdgeView([(13, 20), (13, 18), (20, 22)])

In [13]:
#show_graph(Link_MG, 'Link_MG.html', False, nb)

nt = Network('1000px','1000px', notebook=nb)

# populates the nodes and edges data structures
nt.from_nx(Link_MG)
nt.set_options("""
var options = {
  "physics": {
    "repulsion": {
      "springLength": 325,
      "springConstant": 0
    },
    "minVelocity": 0.75,
    "solver": "repulsion"
  },
  "nodes" : {
      "font" : {
          "color" : "black"
      },
      "borderWidth" : 2,
      "color" : {
          "border" : "black",
          "background" : "white"
        
      }
  }
}
"""
)

nt.show('Link_MG_1.html')

# Построение графа переходов (всевозможных атак)

In [14]:
#CtxA
def get_asset(df, col_name, val):
    return df[df[col_name] == val]
#CtxN
def get_all_assets(df, col_name):
    return df[col_name].unique()
#Pr(a)
def get_pr_asset(df, col_name, val, pr_col_name):
    sub_df = df[df[col_name] == val]
    if len(sub_df):
        return df[df[col_name] == val][pr_col_name].max()
    else:
        return 0
#PhAdj(a)
def phys_adj(G, node):
    return [n for n in G.neighbors(node) if G.edges[node,n]['weight'] == np.inf]

In [15]:
G_dash = nx.MultiDiGraph()

In [16]:
#инциализация графа начальным контекстом
G_dash.add_nodes_from(list(Ctx_0.index))
#nx.set_node_attributes(G_dash, Ctx_0.to_dict(orient='index'))
#вспомогательный DataFrame для более удобных выборок врешин (возможно не надо писать атрибуты в граф)
A_df = pd.DataFrame(Ctx_0)

#инициализация переменного контекста
Ctx = pd.DataFrame(Ctx_0)

In [17]:
%%time
#Основной цикл
while len(Ctx.index):

    #извлечь вершину из текущего контекста
    a_begin = Ctx.iloc[-1]
    Ctx = Ctx.drop([a_begin.name])
    if a_begin['AV'] == np.inf:
        a_end_list = phys_adj(Link_MG,a_begin['a_id'])
        a_end_list.append(a_begin['a_id'])
        for a_end_id in a_end_list:
            a_name = assets.loc[a_end_id].title
            pr_a = get_pr_asset(A_df, 'a_id', a_end_id, 'PR')
            #v_index = list(vuln_matrix[vuln_matrix[a_name] == 1].index)
            #v_sub_df = vulner.loc[v_index]
            v_sub_df = vulners_assets_dict[a_end_id]
            v_sub_df = v_sub_df[v_sub_df.PR <= pr_a]
            for v_id, v_data in v_sub_df.iterrows():
                str_search = "(a_id == {a_id}) & (AV == {v_AV}) & (PR == {v_PR}) & (I == {I})".format(
                    a_id=a_end_id, 
                    v_AV = v_data['s.AV'], 
                    v_PR = v_data['s.PR'], 
                    I = v_data['Imp']
                )
                a_ex_ind = A_df.query(str_search).index
                #не найдена такая вершина в графе
                if len(a_ex_ind) == 0:
                    a_dict = {'a_id' : a_end_id, 'AV' : v_data['s.AV'], 'PR' : v_data['s.PR'], 'I' : v_data['Imp']}
                    A_df = A_df.append(a_dict, ignore_index = True)
                    a_series = A_df.iloc[-1]
                    a_ex_ind = a_series.name
                    G_dash.add_node(int(a_ex_ind))
                    #!!!!!!!!прекращение дальнейшего развития атаки с *конкретного актива* при нанесении ущерба!!!!!!!
                    if a_series['I'] == 0:
                        Ctx = Ctx.append(A_df.iloc[a_ex_ind])
                else:
                    a_ex_ind = a_ex_ind[0]
                G_dash.add_edge(int(a_begin.name), int(a_ex_ind), label = v_id)
    else:
        for a_end_id in Link_MG.nodes():
            if nx.algorithms.shortest_paths.generic.has_path(Link_MG, source = a_begin['a_id'], target = a_end_id):
                d = nx.shortest_path_length(Link_MG, source = a_begin['a_id'], target = a_end_id, weight = 'weight')
            else:
                d = np.inf
            a_name = assets.loc[a_end_id].title
            pr_a = get_pr_asset(A_df, 'a_id', a_end_id, 'PR')
            #v_index = list(vuln_matrix[vuln_matrix[a_name] == 1].index)
            #v_sub_df = vulner.loc[v_index]
            v_sub_df = vulners_assets_dict[a_end_id]
            v_sub_df = v_sub_df[(v_sub_df.PR <= pr_a) & (v_sub_df.AV >= a_begin['AV'] + d)]
            for v_id, v_data in v_sub_df.iterrows():
                str_search = "(a_id == {a_id}) & (AV == {v_AV}) & (PR == {v_PR}) & (I == {I})".format(
                    a_id=a_end_id, 
                    v_AV = v_data['s.AV'], 
                    v_PR = v_data['s.PR'], 
                    I = v_data['Imp']
                )
                a_ex_ind = A_df.query(str_search).index
                #не найдена такая вершина в графе
                if len(a_ex_ind) == 0:
                    a_dict = {'a_id' : a_end_id, 'AV' : v_data['s.AV'], 'PR' : v_data['s.PR'], 'I' : v_data['Imp']}
                    A_df = A_df.append(a_dict, ignore_index = True)
                    a_series = A_df.iloc[-1]
                    a_ex_ind = a_series.name
                    G_dash.add_node(int(a_ex_ind))
                    #!!!!!!!!прекращение дальнейшего развития атаки с *конкретного актива* при нанесении ущерба!!!!!!!                    
                    if a_series['I'] == 0:
                        Ctx = Ctx.append(A_df.iloc[a_ex_ind])
                else:
                    a_ex_ind = a_ex_ind[0]
                G_dash.add_edge(int(a_begin.name), int(a_ex_ind), label = v_id)

Wall time: 110 ms


In [18]:
def mk_title(s):
    if np.isinf(s['AV']):
        s['label'] = '(' + str(int(s['a_id'])) + ', ' + str(s['AV']) + ', ' + str(int(s['PR'])) + ', ' + str(int(s['I'])) + ')'
    else:
        s['label'] = '(' + str(int(s['a_id'])) + ', ' + str(int(s['AV'])) + ', ' + str(int(s['PR'])) + ', ' + str(int(s['I'])) + ')'
    return s
A_df['label'] = ''
A_df = A_df.apply(mk_title, axis = 1)

In [19]:
A_df

,a_id,PR,AV,I,label
0,13,0,2,0,"(13, 2, 0, 0)"
1,13,0,2,1,"(13, 2, 0, 1)"
2,13,0,1,1,"(13, 1, 0, 1)"
3,18,0,2,1,"(18, 2, 0, 1)"
4,18,0,1,1,"(18, 1, 0, 1)"
5,18,2,0,0,"(18, 0, 2, 0)"
6,20,0,2,1,"(20, 2, 0, 1)"
7,20,0,1,1,"(20, 1, 0, 1)"
8,22,0,2,1,"(22, 2, 0, 1)"
9,22,0,1,1,"(22, 1, 0, 1)"


In [20]:
A_df['color'] = 'blue'
A_df.loc[Ctx_0.index,'color'] = 'green'
A_df.loc[A_df.I == 1,'color'] = 'red'
A_df['shape'] = 'ellipse'
nx.set_node_attributes(G_dash, A_df.to_dict(orient='index'))

In [21]:
#nb = False
nt = Network('1000px','1000px', directed=True, notebook=nb)

nt.from_nx(G_dash)
#nt.show_buttons(filter_='physics')
nt.set_options("""
var options = {
  "physics": {
    "repulsion": {
      "springLength": 325,
      "springConstant": 0
    },
    "minVelocity": 0.75,
    "solver": "repulsion"
  },
  "nodes" : {
      "font" : {
          "color" : "white"
      },
      "borderWidth" : 2,
      "color" : {
          "background" : "white"
      }
  }
}
"""
)
nt.show('G_dash_2.html')

# Поиск оптимальной стратегии атаки

## Построение минимального остова на графе переходов

$w^{*}(v) = \sum_{i=1}^{5}N^{5-i}w_i(e)$

### Вычисление весовой функции

In [22]:
G_dash.nodes

a = np.array([[1,1,2,3,5,10], [100,1000,1000,1000,10000,10000]])
b = np.reshape(rankdata(a,'dense',axis=None),a.shape)

In [23]:
L = len(G_dash.nodes)
N = L - 1#количество ребер остова всегда на 1 меньше числа вершин
node_list = list(G_dash.nodes)
#словарь для перевода id вершины в номер
n_id_dict = dict(zip(node_list,range(len(node_list))))
#трехмерный массив для хранения векторной весовой функции (5 матриц весов)
n_matrix = np.zeros((5,L,L))

In [58]:
#установка параметра оптимизации
#проходим по всем ребрам графа (e = (u,v))
for (u,v,d) in list(G_dash.edges):
    #id уязвимости, ассоциированной с ребром
    v_id = G_dash.edges[u,v,d]['label']
    v_m_id = v_id_dict[v_id]
    #id актива (целевого)
    a_id = A_df.at[v,'a_id']
    a_m_id = a_id_dict[a_id]
    #id вершин, из которых собраны ребра 
    u_node_id = n_id_dict[u]
    v_node_id = n_id_dict[v]
    
    #критерий ранга 1 (вероятность успешной реализации)
    if vulners_assets_dict[a_id].at[v_id,'P_v'] == 0:
        P_v = np.inf
    else:
        P_v = -np.log(vulners_assets_dict[a_id].at[v_id,'P_v'])
    n_matrix[0, u_node_id, v_node_id] = P_v
    
    #критерий ранга 2 (вероятность необнаружения атаки)
    if vulners_assets_dict[a_id].at[v_id,'P_d'] == 0:
        P_d = np.inf
    else:
        P_d = -np.log(vulners_assets_dict[a_id].at[v_id,'P_d'])
    n_matrix[1, u_node_id, v_node_id] = P_d
    
    #критерий ранга 3 (ожидаемое время обнаружения)
    tau = matrix_measures[a_m_id,v_m_id].min()
    n_matrix[2, u_node_id, v_node_id] = tau
    
    #критерий ранга 4 - время восстановления
    TTR = assets.at[a_id,'TTR']
    n_matrix[3, u_node_id, v_node_id] =TTR

    #критерий ранга 5 - время атаки
    t = vulners_assets_dict[a_id].at[v_id,'t']
    n_matrix[4, u_node_id, v_node_id] = t

In [59]:
#критерий 2 (вероятность необнаружения) - заменяем веса ребер на их ранг по возрастанию
n_matrix[1] = np.reshape(rankdata(n_matrix[1],'dense',axis=None),n_matrix[1].shape)
#критерий 3 (ожидаемое время обнаружения) - заменяем веса ребер на их ранг по возрастанию
n_matrix[2] = np.reshape(rankdata(n_matrix[2],'dense',axis=None),n_matrix[2].shape)

#вычисляем степенную весовую функцию L^(ранг веса ребра)
n_matrix[1] = np.power(L, n_matrix[1])
n_matrix[2] = np.power(L, n_matrix[2])


In [60]:
#нормализуем отдельные компоненты векторного веса
for i in range(5):
    min_val = n_matrix[i].min()
    max_val = n_matrix[i].max()
    n_matrix[i] = (n_matrix[i] - min_val)/(max_val - min_val)
    

In [65]:
#считаем скалярную весовую функцию
for (u,v,d) in list(G_dash.edges):
    u_node_id = n_id_dict[u]
    v_node_id = n_id_dict[v]
    G_dash.edges[u,v,d]['weight'] = n_matrix[4, u_node_id, v_node_id] + n_matrix[3, u_node_id, v_node_id]*N + n_matrix[2, u_node_id, v_node_id]*(N**2) + n_matrix[1, u_node_id, v_node_id]*(N**3) + n_matrix[0, u_node_id, v_node_id]*(N**4)
    G_dash.edges[u,v,d]['title'] = "w: {}".format(G_dash.edges[u,v,d]['weight'])


### Решение задачи минимального остова

In [68]:
ctx_nodes = list(Ctx_0.index)

In [69]:
%%time
ids = A_df[A_df.I == 1]['a_id'].unique()
for a_id in ids:
    #fin_node = 'fin: ' + str(a_id)
    fin_node = assets.at[a_id,'title'] + ' ' + str(a_id)
    G_dash.add_node(fin_node)
    adj_fin = list(A_df[(A_df.a_id == a_id) & (A_df.I == 1)].index)
    for fin_a_id in adj_fin:
        G_dash.add_edge(fin_a_id, fin_node, weight = 0)
   

Wall time: 2.99 ms


In [70]:
nt = Network('750','750px', directed=True, notebook=nb)
nt.set_options("""
  var options = {
      "layout": {
        "hierarchical": {
          "enabled": false
          
        }
      }
    }
"""
)
nt.from_nx(G_dash)
nt.show('B_3.html')

In [71]:
%%time
B = nx.DiGraph()
root_node = ctx_nodes[0]
for root_node in ctx_nodes:
    G_dash_temp = G_dash.copy()
    del_nodes = set(ctx_nodes) - set([root_node])
    G_dash_temp.remove_nodes_from(del_nodes)
    G_dash_temp = G_dash_temp.subgraph(nx.shortest_path(G_dash_temp,root_node)).copy()
    #G_dash_temp.add_node('sart')
    G_dash_temp.add_edge('start', root_node, weight = 0)
    B_part = nx.DiGraph(nx.algorithms.tree.branchings.minimum_spanning_arborescence(G_dash_temp,attr='weight', default=0, preserve_attrs=True))
    B = nx.algorithms.operators.binary.compose(B, B_part)

    

Wall time: 2.99 ms


In [72]:
for n in list(B.nodes()):
    if n in A_df.index:
        B.nodes[n]['color'] = A_df.at[n,'color']

In [73]:
nt = Network('750','750px', directed=True, notebook=nb)
nt.set_options("""
  var options = {
      "layout": {
        "hierarchical": {
          "enabled": true,
          "sortMethod": "directed"
        }
      }
    }
"""
)
nt.from_nx(B)
nt.show('B_4.html')

## Редуцирование минимального остова до наиболее выгодных направлений атак на активы

In [74]:
%%time
was_del = True
idx = list(A_df.index)
while was_del:
    was_del = False
    for i in idx:
        if (i in B) and (len(B.edges(i)) == 0):
            B.remove_node(i)
            was_del = True

Wall time: 0 ns


In [75]:
%%time
length = nx.single_source_dijkstra_path_length(B, 'start')

Wall time: 0 ns


In [76]:
%%time
for n, l in length.items():
    #для старой формулы (логарифмы)
    #raw_l = np.exp(-l)
    #для новой формулы
    raw_l = np.exp(-l)
    
    B.nodes[n]['raw_length'] = raw_l
    B.nodes[n]['length'] = l
    B.nodes[n]['title'] = 'w: ' + str(l) + ' rw: ' + str(raw_l)

Wall time: 0 ns


In [78]:
nt = Network('750','750px', directed=True, notebook=nb)

 
nt.from_nx(B)
nt.set_options("""
  var options = {
      "layout": {
        "hierarchical": {
          "enabled": true,
          "sortMethod": "directed"
        }
      }
    }
"""
)
nt.show('B_reduce1_5.html')

## Поиск оптимального разреза дерева неисправностей (минимум элементов, максимум вероятности успеха незаметной атаки)


In [79]:
%%time
def add_FT_nodes(tree, parent, child, level):
    d_type = {
        'basic': 0,
        'or': 0,
        'root': 1
    }
    c_node = child['name']
    #добавляем новую вершину дерева
    tree.add_node(c_node, impact=child.get('impact', 0), level = level)
    #tree.add_node(c_node, impact=child.get('impact', 0))
    #смотрим на тип вершины и переводим в потерю
    if child['node_type'] == 'k_from_n':
        #мажоритарное или - потеря вершины = k - 1
        tree.nodes[c_node]['loss'] = int(child['k']) - 1
    else:
        if child['node_type'] == 'and':
            tree.nodes[c_node]['loss'] = len(child['childs']) - 1
        else:
            #в прочих случаях берем по словарю (или - 0, и - 1, по умолчанию - 0)
            tree.nodes[c_node]['loss'] = d_type.get(child['node_type'], 0)
    tree.nodes[c_node]['title'] = 'тип: {node_type} k: {loss} ущерб: {impact}'.format(
        node_type = child['node_type'], 
        loss = tree.nodes[c_node]['loss'],
        impact = tree.nodes[c_node]['impact']
    )
    if (child['node_type'] == 'k_from_n') or (child['node_type'] == 'or'):
        tree.nodes[c_node]['shape'] = 'triangleDown'
    elif (child['node_type'] == 'and'):
        tree.nodes[c_node]['shape'] = 'triangle'
    else:
        tree.nodes[c_node]['shape'] = 'ellipse'
       
        
        
    #если  не корень (parent отличается от None)
    #добавляем ребро от потомка к предку (для последующего поиска потока)
    #у всех ребер пропускная способность = 1
    if parent:
        tree.add_edge(c_node, parent, capacity=1)
    #проходим рекурентно по всем детям
    for c in child.get('childs', []):
        #рекуррентно вызываем создание узлов
        tree = add_FT_nodes(tree, c_node, c, level + 1)
    return tree
#чтение файла с json описанием дерева неисправностей
jf = open('.\\' + input_data + '\\FT.json','r',encoding='utf-8')
FT_j = json.load(jf)

FT = nx.DiGraph()
#if len(FT_j) > 1:
#    print('Неверное дерево неисправностей: более одного корня!')
#    exit(1)
FT = add_FT_nodes(FT, None, FT_j, 0)
root_name = FT_j['name']
FT_orig = FT.copy()
FT_orig.nodes[root_name]['loss'] = 0

NameError: name 'input_data' is not defined

In [ ]:

nt = Network('750','750px', directed=True, notebook=nb)
nt.set_options("""
  var options = {
      "layout": {
        "hierarchical": {
          "enabled": true,
          "sortMethod": "directed",
          "direction" : "UD"
        }
      }
    }
"""
)

nt.from_nx(FT)


nt.show('FT_full_6.html')

In [ ]:
#добавляем сток для всех вентилей (вершины с ненулевыми потерями)
#добавляем исток для всех листовых вершин
FT.add_node('s', color='green')
FT.add_node('t', color = 'green')
for n in FT.nodes:
    #пропускаем добавленные s и t
    if (n != 's') and (n != 't'):
        #если лист - то нужно добавить ребро от s к n
        if FT.degree(n) - len(FT.edges(n)) == 0:
            #TODO: необходимо добавить стоимость (для нахождения не просто минимального, но и оптимального набора вершин)
            if n in B.nodes():
                #ВАЖНО: алгоритм поиска максимального потока минимальной стоимости требует целочисленные веса (стоимости)
                weight = int(np.round(B.nodes[n]['length'] * 1000))
            FT.add_edge('s', n, capacity = 1 , weight = weight)
        if int(FT.nodes[n]['loss']) > 0:
            FT.add_edge(n,'t',capacity = int(FT.nodes[n]['loss']))


In [ ]:
nt = Network('750','750px', directed=True, notebook=nb)

nt.from_nx(FT)
#nt.show_buttons(filter_='physics')
nt.set_options("""
var options = {
  "physics": {
    "repulsion": {
      "springLength": 325,
      "springConstant": 0
    },
    "minVelocity": 0.75,
    "solver": "repulsion"
  }
}
"""
)
nt.show('G_dash_7.html')

In [ ]:
%%time
flow_dict = nx.algorithms.flow.max_flow_min_cost(FT, s = 's', t = 't')

In [ ]:
for (s_node, n_dict) in flow_dict.items():
    for (t_node, flow) in n_dict.items():
        FT.edges[s_node,t_node]['flow'] = flow
        if flow:
            FT.edges[s_node,t_node]['color'] = 'red'

In [ ]:
nt = Network('750','750px', directed=True, notebook=nb)

nt.from_nx(FT)
#nt.show_buttons(filter_='physics')
nt.set_options("""
var options = {
  "physics": {
    "repulsion": {
      "springLength": 325,
      "springConstant": 0
    },
    "minVelocity": 0.75,
    "solver": "repulsion"
  }
}
"""
)
nt.show('G_dash_8.html')

In [ ]:
FT.remove_node('s')
FT.remove_node('t')
for e in list(FT.edges()):
    if FT.edges[e].get('flow',0) == 0:
        FT.remove_edge(*e)


In [ ]:
nt = Network('750','750px', directed=True, notebook=nb)

nt.from_nx(FT)
#nt.show_buttons(filter_='physics')
nt.set_options("""
var options = {
  "physics": {
    "repulsion": {
      "springLength": 325,
      "springConstant": 0
    },
    "minVelocity": 0.75,
    "solver": "repulsion"
  }
}
"""
)
nt.show('G_dash_9.html')

In [ ]:
%%time
FT_reduce = FT.subgraph(nx.shortest_path(FT.to_undirected(),root_name))

In [ ]:
nt = Network('750','750px', directed=True, notebook=nb)

nt.from_nx(FT_reduce)
nt.set_options("""
  var options = {
      "layout": {
        "hierarchical": {
          "enabled": true,
          "sortMethod": "directed",
          "direction" : "UD"
        }
      }
    }
"""
)

nt.show('FT_10.html')

In [ ]:
targets = []
for n in FT_reduce.nodes():
     if FT_reduce.degree(n) - len(FT_reduce.edges(n)) == 0:
            targets.append(n)
targets

## Повторное редуцирование остова атак (оставляем только цели из оптимального разреза)

In [ ]:
%%time
was_del = True
while was_del:
    was_del = False
    for i in list(B.nodes()):
        #print(str(i) + '  ' + str(B.edges(i)) + '\n')
        #удаляем все листовые вершины, не попавшие в targets
        if (len(B.edges(i)) == 0) and not(i in targets):
            B.remove_node(i)
            was_del = True

In [ ]:
nt = Network('750','750px', directed=True, notebook=nb)

nt.from_nx(B)
nt.set_options("""
  var options = {
      "layout": {
        "hierarchical": {
          "enabled": true,
          "sortMethod": "directed",
          "direction" : "UD"
        }
      }
    }
"""
)
nt.show('B_reduce2_11.html')


# Симуляция процесса атаки по оптимальному сценарию

In [ ]:
def mk_attack_path(B, Ctx, target):
    pathes = nx.multi_source_dijkstra_path(B,Ctx)
    return pathes.get(target, [])[1:]

In [ ]:
sym_log = []
def log_msg(timestamp, msg_dict):
    global sym_log
    sym_log.append((timestamp, msg_dict))
    return

def print_msg():
    return

msg_type = {
    1 : {
        'prefix' : 'ATTACK Success',
        'desc' : 'Завершена атака в отношении всех доступных активов. Общий ущерб: {impact}'
    },
    2 : {
        'prefix' : 'ATTACK Failed',
        'desc' : 'Присутствие злоумышленника устранено на всех активах. Общий ущерб: {impact}'
        },
    3 : {
        'prefix' : 'ATTACK Failed',
        'desc' : 'Система остановлена на внеплановое обслуживание из-за действий нарушителя. Общий ущерб: {impact}',
    },
    4 : {
        'prefix' : 'INFO',
        'desc' : 'bogus'
    },
    5 : {
        'prefix' : 'ATTACK Started',
        'desc' : 'Начата эксплуатация уязвимости {v_id} в отношении актива {a_name}',
    },
    6 : {
        'prefix' : 'ATTACK Success',
        'desc' : 'Успешно завершена эксплуатация уязвимости {v_id} в отношении актива {a_name}',
    },
    7 : {
        'prefix' : 'ATTACK Detected',
        'desc' : 'Обнаружена эксплуатация уязвимости {v_id} в отношении актива {a_name}. Процесс восстановления займет {TTR} шагов',
    },
    8 : {
        'prefix' : 'IMPACT Applied',
        'desc' : 'Нанесен ущерб активу {a_name}. Текущий уровень ущерба: {impact}'
    },
    9 : {
        'prefix' : 'ATTACK Failed',
        'desc' : 'Эксплуатация уязвимости {v_id} в отношении актива {a_name} завершилась неуспешно',
    },
    10 : {
        'prefix' : 'ATTACK Detected',
        'desc' : 'Обнаружена эксплуатация уязвимости {v_id} с источника атаки {a_name}. Процесс восстановления займет {TTR} шагов'
    },
    11 : {
         'prefix' : 'IMPACT Detected',
         'desc' : 'Обнаружен скомпрометированный актив {a_name}. Процесс восстановления займет {TTR} шагов'
    }
}
    



In [ ]:
def count_FT(FT, node, values):
    #если число входящих не 0, то это вентиль или корень
    if len(FT.in_edges(node)):
        #проходим по всем подчиненным вершинам вентиля
        level = 0
        impact = 0
        for suc in FT.predecessors(node):
            (delta_impact, res) = count_FT(FT, suc, values)
            level = level + res
            #для каждого отказавшего узла суммируем ущерб. Соответственно, если для более высокоуровневого узла ущерб не достигнут, 
            #то будет возвращена сумма по всем достигнутым ущербам
            impact = impact + delta_impact
        if (level > FT.nodes[node]['loss']):
            return (FT.nodes[node].get('impact',0), 1)
        else:
            return (impact, 0)
    #если это листовая вершина (элементарное событие)
    else:
        if int(node in values['title'].values):
            return(FT.nodes[node].get('impact',0), 1)
        else:
            return(0, 0)




In [ ]:
def sim_attack(B, T, N, Ctx_0, targets):

    Ctx = Ctx_0.copy()
    AIP = pd.Series(dtype=object)
    path = []
    total_impact = 0
    cur_target = None
    impact = pd.DataFrame(columns=['node', 'v_id', 'v_t', 'title', 'P_d'])
    val = [list(B.pred[n].keys())[0] for n in targets]



    for t in range(1,T + 1, 1):
        #условия прекращения симуляции:
        #val = [] - все цели поражены
        if (len(val) == 0):
            #Завершена атака в отношении всех доступных активов. Общий ущерб: {impact}
            log_msg(t, {'type' : 1, 'params' : {'impact' : total_impact}})
            break
        #Ctx = [] - нарушитель побежден (не осталось никакого контекста)
        if (len(Ctx) == 0):
            #Присутствие злоумышленника устранено на всех активах. Общий ущерб: {impact}
            log_msg(t, {'type' : 2, 'params' : {'impact' : total_impact}})
            break
        #N < 0 - кончились "жизни"
        if N <= 0:
            #Система остановлена на внеплановое обслуживание из-за действий нарушителя. Общий ущерб: {impact}
            log_msg(t, {'type' : 3, 'params' : {'impact' : total_impact}})
            break


        #нет выбранной цели (начало атаки, завершили атаку на цель или изменился контекст в результате работы защитных мер)
        if not(cur_target):
            path = []
            while not(len(path)):
                cur_target = val[0]
                path = mk_attack_path(B, Ctx, cur_target)
                if not(len(path)):
                    val.pop()
                    cur_target = None
                if not(len(val)) and not(cur_target):
                    break
            if not(cur_target):
                break
        #нет активной попытки атаки
        if (len(AIP) == 0):
            #берем, но не выкидываем (если атака будет неуспешной, то нужно повторять ее)
            AIP['node'] = path[0]
            AIP['v_id'] = list(B.in_edges(AIP['node'], data='label'))[0][2]
            AIP['v_t'] = vulner.at[AIP['v_id'],'t']
            AIP['title'] = assets.at[A_df.at[AIP['node'],'a_id'],'title']
            #Начата эксплуатация уязвимости {v_id} в отношении актива {a_name}
            log_msg(t,{'type' : 5, 'params' : {'v_id' : AIP['v_id'], 'a_name' : AIP['title']}})
        else:
            AIP['v_t'] = AIP['v_t'] - 1
        #время текущей атаки завершилось - проверяем результат
        if AIP['v_t'] == 0:
            #бросок кубика на успех атаки
            if vulner.at[AIP['v_id'],'P_v'] > random.random():
                #Успешно завершена эксплуатация уязвимости {v_id} в отношении актива {a_name}
                log_msg(t,{'type' : 6, 'params' : {'v_id' : AIP['v_id'], 'a_name' : AIP['title']}})
                AIP['P_d'] = vulner.at[AIP['v_id'],'P_d']
                Ctx.append(AIP['node'])
                #если дошли до целевого ущерба
                if AIP['node'] == cur_target:
                    impact = impact.append(AIP, ignore_index = True)
                    val.remove(cur_target)
                    path = []
                    cur_target = None
                    (cur_impact, flag) = count_FT(FT_orig, root_name, impact)
                    total_impact = max(total_impact, cur_impact)
                    #Нанесен ущерб активу {a_name}. Текущий уровень ущерба: {impact}
                    log_msg(t,{'type' : 8, 'params' : {'a_name' : AIP['title'], 'impact' : total_impact}})
                else:
                    junk = path.pop(0)
                #бросок кубика на обнаружение атаки
                if AIP['P_d'] > random.random():
                    #запуск восстановления, если ранее не было запущено
                    if assets.at[A_df.at[AIP['node'],'a_id'],'t_recover'] == 0:
                        assets.at[A_df.at[AIP['node'],'a_id'],'t_recover'] = t
                    #Обнаружена эксплуатация уязвимости {v_id} в отношении актива {a_name}. Процесс восстановления займет {TTR} шагов
                    log_msg(t,{'type' : 7, 'params' : {'v_id' : AIP['v_id'], 'a_name' : AIP['title'], 'TTR' : assets.at[A_df.at[AIP['node'],'a_id'],'TTR']}})
                    N = N - 1
                #только при успешной атаке мы убираем вершину из пути

            #атака неуспешна (бросок кубика не в пользу нарушителя)
            else:
                #Эксплуатация уязвимости {v_id} в отношении актива {a_name} завершилась неуспешно
                log_msg(t,{'type' : 9, 'params' : {'v_id' : AIP['v_id'], 'a_name' : AIP['title']}})
                atk_source =  list(B.pred[AIP['node']].keys())[0]
                if vulner.at[AIP['v_id'],'P_d'] > random.random():
                    #запуск восстановления только, если ранее не было запущено
                    if assets.at[A_df.at[atk_source,'a_id'],'t_recover'] == 0:
                        assets.at[A_df.at[atk_source,'a_id'],'t_recover'] = t
                    #Обнаружена эксплуатация уязвимости {v_id} с источника атаки {a_name}. Процесс восстановления займет {TTR} шагов
                    log_msg(t,{'type' : 10, 'params' : {'v_id' : AIP['v_id'], 'a_name' : assets.at[A_df.at[atk_source,'a_id'],'title'], 'TTR' : assets.at[A_df.at[atk_source,'a_id'],'TTR']}})
                    N = N - 1
            #сброс текущей цели атаки (для выбора следующей цели)
            AIP = pd.Series(dtype=object)
        #проходим по всем вершинам контекста в поисках тех, где сработал таймер восстановления (или случился детект)
        for ctx_node in Ctx:
            a_id = A_df.at[ctx_node,'a_id']
            asset = assets.loc[a_id]
            #если активу нанесен ущерб (каждый ход можем это обнаружить)
            if (ctx_node in impact['node'].values) and (asset['t_recover'] == 0):
                impact_node = impact.loc[impact['node'] == ctx_node]
                impact_node = impact_node.iloc[0]
                if impact_node['P_d'] > random.random():
                    assets.at[A_df.at[ctx_node,'a_id'],'t_recover'] = t
                    #Обнаружен скомпрометированный актив {a_name}. Процесс восстановления займет {TTR} шагов
                    log_msg(t,{'type' : 11, 'params' : {'a_name' : impact_node['title'], 'TTR' : assets.at[A_df.at[ctx_node,'a_id'],'TTR']}})
                    N = N - 1
            #завершено восстановление актива
            if (asset['t_recover'] > 0) and (t - asset['t_recover'] >= asset['TTR'] ):
                assets.at[a_id,'t_recover'] = 0
                #удаляем из набора impact
                idx = list(impact[impact['node'] == ctx_node].index)
                impact.drop(labels=idx, axis=0, inplace=True)
                val.extend(idx)
                #удаляем все вхождения в конекст
                idx = list(A_df[A_df['a_id'] == a_id].index)
                for idx_iter in idx:
                    if idx_iter in Ctx:
                        Ctx.remove(idx_iter)
                #после изменения контекста некоторые цели могли стать недоступны - нужно перестроить граф
                #и убрать лишнее из массива val
                sub_nodes = set()
                for b_node in Ctx:
                    sub_nodes.union(set(nx.shortest_path(B.to_undirected(),b_node)))
                B = B.subgraph(list(sub_nodes))
                val = []
                for n in targets:
                    if n in B:
                        val.append(list(B.pred[n].keys())[0])
                path = []
                cur_target = None
                AIP = pd.Series(dtype=object)
            #обход периодических мер
            for (t_idx, t_measure) in t_measures.iterrows():
                #прошел период
                if np.mod(t,t_measure['Период']) == 0:
                    #обходим вершины в контексте
                    for ctx_node in Ctx:
                        a_id = A_df.at[ctx_node,'a_id']
                        asset = assets.loc[a_id]
                        #если мера обнаружила компрометированный актив
                        if t_measure['P_d'] > random.random():
                            assets.at[a_id,'t_recover'] = t
                            #Обнаружен скомпрометированный актив {a_name}. Процесс восстановления займет {TTR} шагов
                            log_msg(t,{'type' : 11, 'params' : {'a_name' : assets.at[A_df.at[ctx_node,'a_id'],'title'], 'TTR' : assets.at[A_df.at[ctx_node,'a_id'],'TTR']}})
                            N = N - 1
    return (t, total_impact)


In [ ]:
%%time
num_sim = 100



#исходные данные для симуляции:
#B - уже оптимизированное дерево
sym_log = []
#максимальное время симуляции
T = 1000
#уровень бдительности ("жизни")
N = 5
#Конекст на дереве атак
Ctx_0 = list(B.succ['start'].keys())
#targets - массив предков целевых вершин (листьев) - так листья добавлялись для поиска оптимальных путей


res = {}

for i in range(0,num_sim):
    #(t, impact) = sim_attack(B, T, N, Ctx_0, targets)
    (t, impact) = sim_attack(G_dash, T, N, Ctx_0, targets)
    if impact in res.keys():
        res[impact] = [res[impact][0] + 1, res[impact][1] + t]
    else:
        res[impact] = [1, t]


In [ ]:
#считаем среднее время атаки для каждого уровня ущерба
for (key, val) in res.items():
    res[key][1] = val[1]/val[0]

In [ ]:
res

In [ ]:
sym_log